In [3]:
# 1. KURULUM VE KÜTÜPHANELER
!pip install groq

import groq
import requests  # Hava durumu ve Döviz API'leri için
import re
import math
import time      # Benchmark (hız testi) yapmak için
from collections import Counter
from google.colab import userdata # API Key güvenliği için
from groq import Groq

print(f" Kurulum tamamlandı! Groq versiyonu: {groq.__version__}")

 Kurulum tamamlandı! Groq versiyonu: 1.0.0


In [75]:
import os
import re
import math
import requests
from collections import Counter
from groq import Groq

# =============================================================================
# 1. KURULUM VE API AYARLARI
# =============================================================================
groq_api_key = "GROQ_API_KEY"

try:
    client = Groq(api_key=groq_api_key)
except Exception as e:
    print(f"HATA: API bağlantısı kurulamadı: {e}")

class CostTracker:
    def __init__(self):
        self.input_price_1m = 0.59
        self.output_price_1m = 0.79
        self.total_cost_usd = 0.0

    def update(self, input_tok, output_tok):
        step_cost = (input_tok * (self.input_price_1m / 1_000_000)) + \
                    (output_tok * (self.output_price_1m / 1_000_000))
        self.total_cost_usd += step_cost
        return step_cost

tracker = CostTracker()

# =============================================================================
# 2. RAG MOTORU
# =============================================================================
class SimpleRAG:
    def __init__(self):
        self.vector_store = []

    def _normalize(self, text):
        tr_map = str.maketrans("çğıöşüÇĞİÖŞÜ", "cgiosuCGIOSU")
        text = text.translate(tr_map).lower()
        return text.replace("limitleri", "limit").replace("limiti", "limit")

    def get_embedding(self, text):
        normalized_text = self._normalize(text)
        words = re.findall(r'\w+', normalized_text)
        return Counter(words)

    def _cosine_similarity(self, vec1, vec2):
        intersection = set(vec1.keys()) & set(vec2.keys())
        numerator = sum([vec1[x] * vec2[x] for x in intersection])
        sum1 = sum([vec1[x]**2 for x in vec1.keys()])
        sum2 = sum([vec2[x]**2 for x in vec2.keys()])
        denominator = math.sqrt(sum1) * math.sqrt(sum2)
        return float(numerator) / denominator if denominator else 0.0

    def ingest(self, text):
        chunks = [s.strip() for s in text.split('\n') if s.strip()]
        for chunk in chunks:
            self.vector_store.append({
                "content": chunk,
                "vector": self.get_embedding(chunk)
            })

    def retrieve(self, query):
        query_vec = self.get_embedding(query)
        scores = []
        for item in self.vector_store:
            score = self._cosine_similarity(query_vec, item["vector"])
            scores.append((score, item["content"]))
        scores.sort(key=lambda x: x[0], reverse=True)
        return scores[0][1] if scores and scores[0][0] > 0.01 else "Bilgi bulunamadı."

rag_engine = SimpleRAG()
rag_engine.ingest("""
UCAK TIPI: BOEING 737 - Maksimum ruzgar limiti 35 km/s - Inis gorus mesafesi 800 metre
UCAK TIPI: AIRBUS A320 - Maksimum ruzgar limiti 38 km/s - Inis gorus mesafesi 500 metre
UCAK TIPI: CESSNA 172 - Maksimum ruzgar limiti 15 km/s - Yogun yagmur yasak
YAKIT FIYATI: Havacılık yakıtı (Avgas) galon fiyatı 6 USD
""")

# =============================================================================
# 3. ARAÇLAR
# =============================================================================

def calculator(expression: str) -> str:
    try:
        clean_expr = re.sub(r'[^0-9+\-*/().]', '', expression)
        return str(eval(clean_expr))
    except: return "Hesaplama Hatası"

def search_knowledge_base(query: str) -> str:
    print(f"\n [RAG SCAN] '{query}' aranıyor...")
    return rag_engine.retrieve(query)

def get_weather(city: str) -> str:
    print(f"\n [WEATHER API] {city} verisi...")
    try:
        url = f"https://geocoding-api.open-meteo.com/v1/search?name={city}&count=1"
        res = requests.get(url, timeout=5).json()
        if not res.get("results"): return f"HATA: {city} şehri bulunamadı."
        lat, lon = res["results"][0]["latitude"], res["results"][0]["longitude"]
        w_url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true"
        w = requests.get(w_url, timeout=5).json()["current_weather"]
        return f"Ruzgar hizi: {w['windspeed']} km/s"
    except: return "HATA: Hava durumu servislerine ulaşılamıyor."

def get_exchange_rate(currency_code: str) -> str:
    print(f"\n [FINANCE API] {currency_code}...")
    try:
        clean_code = currency_code.upper().replace("USD", "").strip()
        res = requests.get("https://open.er-api.com/v6/latest/USD", timeout=5).json()
        rate = res["rates"].get(clean_code)
        return f"1 USD = {rate} {clean_code}" if rate else "Hata: Para birimi bulunamadı."
    except: return "HATA: Kur servisi yanıt vermiyor."

known_actions = {
    "calculator": calculator,
    "search_knowledge_base": search_knowledge_base,
    "get_weather": get_weather,
    "get_exchange_rate": get_exchange_rate
}

# =============================================================================
# 4. AJAN VE SİSTEM PROMPTU
# =============================================================================

system_prompt = f"""
Sen bir Havacılık Dispatcher ajanısın.

MEVCUT ARAÇLAR:
- search_knowledge_base: Teknik limitleri ve yakıt fiyatını bulur. (Örn: 'Boeing 737')
- calculator: Matematiksel hesaplama yapar. (Örn: '1250 * 6')
- get_weather: Hava durumunu çeker. (⚠️ Sadece şehir ismi yaz: 'Berlin')
- get_exchange_rate: Döviz kurunu çeker. (⚠️ Sadece hedef birimi yaz: 'TRY')

⚠️ ZORUNLU KURALLAR:
1. GÜVENLİK PROTOKOLÜ: Önce 'search_knowledge_base' ile teknik limitleri sorgula.
2. TEK ADIM: Her seferinde SADECE BİR aksiyon al. PAUSE ifadesini unutma.
3. BİLGİ SORULARI: Kullanıcı sadece dökümandan bilgi istiyorsa (Scenario A), hava durumu veya kur çekme. Doğrudan döküman bilgisiyle 'Answer:' ver.
4. SONSUZ DÖNGÜ: Eğer bir araç hata verirse, aynı parametreyle tekrar çağırma.
5. GEÇERSİZ ARAÇ: 'Action:' kısmına sadece yukarıdaki 4 araçtan birini yazabilirsin. Kendi kendine araç uydurma.
6. Kullanıcı açıkça bir şehir ismi belirtmedikçe get_weather aracını kullanma. Sadece sorulan soruya dökümandaki verilerle cevap ver.

Örnek Akış:
Thought: Uçağın rüzgar limitini bulmalıyım.
Action: search_knowledge_base: Airbus A320
PAUSE
""".strip()

class ReActAgent:
    def __init__(self):
        self.messages = [{"role": "system", "content": system_prompt}]
        self.prev_actions = set()

    def run(self, user_input, max_turns=10):
        self.messages.append({"role": "user", "content": user_input})

        for _ in range(max_turns):
            completion = client.chat.completions.create(
                model="llama-3.3-70b-versatile",
                messages=self.messages,
                temperature=0,
                stop=["PAUSE", "Observation:"]
            )

            response = completion.choices[0].message.content
            tracker.update(completion.usage.prompt_tokens, completion.usage.completion_tokens)

            print(f"\n\033[94m{response}\033[0m")
            self.messages.append({"role": "assistant", "content": response})

            match = re.search(r'Action:\s*(\w+):\s*(.*)', response, re.IGNORECASE)

            if match:
                action, action_input = match.groups()
                action, action_input = action.strip(), action_input.strip()

                # KRİTİK HATA YÖNETİMİ VE DÖNGÜ KONTROLÜ
                call_key = f"{action}:{action_input}"
                if action not in known_actions:
                    obs = f"HATA: '{action}' geçerli bir araç değil. Lütfen sadece listedeki 4 aracı (calculator, search_knowledge_base, get_weather, get_exchange_rate) kullan veya cevabını 'Answer:' ile bitir."
                elif call_key in self.prev_actions:
                    obs = "HATA: Bu işlemi aynı parametreyle zaten yaptın ve sonuç alamadın. Lütfen farklı bir adım dene veya eldeki verilerle 'Answer:' ver."
                else:
                    self.prev_actions.add(call_key)
                    obs = known_actions[action](action_input)

                print(f"\033[92mObservation: {obs}\033[0m")
                self.messages.append({"role": "user", "content": f"Observation: {obs}"})

            elif "Answer:" in response:
                return response
        return "İşlem süresi doldu."

# =============================================================================
# 5. ÇALIŞTIRMA
# =============================================================================
def ask(question):
    print(f"\n{'='*60}\nSORU: {question}\n{'='*60}")
    ReActAgent().run(question)
    print(f"\n\033[95m[TOPLAM MALİYET] ${tracker.total_cost_usd:.5f}\033[0m")

# Scenario A Testi
ask("Cessna 172 tipi uçaklar için dökümanda belirtilen kısıtlayıcı hava durumu koşulu nedir?")


SORU: Cessna 172 tipi uçaklar için dökümanda belirtilen kısıtlayıcı hava durumu koşulu nedir?

Thought: Cessna 172 için teknik limitleri ve kısıtlayıcı hava durumu koşullarını bulmalıyım.
Action: search_knowledge_base: Cessna 172


 [RAG SCAN] 'Cessna 172' aranıyor...
Observation: UCAK TIPI: CESSNA 172 - Maksimum ruzgar limiti 15 km/s - Yogun yagmur yasak

Thought: Cessna 172 için maksimum rüzgar limiti 15 km/s ve yoğun yağmur uçuşa engel teşkil ediyor.
Answer: Cessna 172 için dökümanda belirtilen kısıtlayıcı hava durumu koşulu, maksimum 15 km/s rüzgar limiti ve yoğun yağmur yasaktır.

[TOPLAM MALİYET] $0.00064


In [76]:
ask("Airbus A320 uçağımızla Berlin'e bir uçuş planlıyoruz. Berlin'deki güncel rüzgar hızı uçağın teknik limitleri dahilinde mi? Eğer güvenliyse, bu uçuş için gereken 1250 galon yakıtın toplam maliyetini güncel Türk Lirası (TRY) cinsinden hesapla.")


SORU: Airbus A320 uçağımızla Berlin'e bir uçuş planlıyoruz. Berlin'deki güncel rüzgar hızı uçağın teknik limitleri dahilinde mi? Eğer güvenliyse, bu uçuş için gereken 1250 galon yakıtın toplam maliyetini güncel Türk Lirası (TRY) cinsinden hesapla.

Thought: Önce Airbus A320 uçağının teknik limitlerini ve yakıt fiyatını bulmalıyım.
Action: search_knowledge_base: Airbus A320


 [RAG SCAN] 'Airbus A320' aranıyor...
Observation: UCAK TIPI: AIRBUS A320 - Maksimum ruzgar limiti 38 km/s - Inis gorus mesafesi 500 metre

Thought: Şimdi Berlin'deki güncel hava durumunu ve rüzgar hızını öğrenmem gerekiyor.
Action: get_weather: Berlin


 [WEATHER API] Berlin verisi...
Observation: Ruzgar hizi: 15.0 km/s

Thought: Berlin'deki rüzgar hızı (15 km/s) Airbus A320 uçağının maksimum rüzgar limitinin (38 km/s) altında olduğu için uçuş güvenlidir. Şimdi 1250 galon yakıtın maliyetini hesaplamak için güncel yakıt fiyatını ve doların Türk Lirası cinsinden değerini bulmam gerekiyor. 
Action: search_knowledge_